In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
train=pd.read_csv("C:/Users/user/Downloads/train_v9rqX0R.csv")
test=pd.read_csv('C:/Users/user/Downloads/test_AbJTz2l.csv')

In [4]:
df = pd.concat([train, test])

In [5]:
df['Item_Fat_Content'].replace({'low fat':'Low Fat','reg':'Regular','LF':'Low Fat'},inplace=True)

In [6]:
#replaces nan values with mode of Outlet_Size
df['Outlet_Size'].fillna('Medium',inplace=True)

In [7]:
df['Item_Weight'].fillna(12.792,inplace=True)

In [8]:
df['Item_Identifier_num'] = df['Item_Identifier'].str[-2:]

In [9]:
df['Item_Identifier_abc'] = df['Item_Identifier'].str[:-2]

In [10]:
df['Item_Visibility_per'] = df['Item_Visibility']*100

In [11]:
df.drop(['Outlet_Identifier','Item_Identifier','Item_Visibility'],axis=1,inplace=True)

In [12]:
dummy_list=['Item_Fat_Content','Item_Type','Item_Identifier_abc','Outlet_Size','Outlet_Type','Outlet_Location_Type','Item_Identifier_abc','Item_Identifier_num']

In [13]:
dummy_drop = []
clean_df = df
for i in dummy_list:
    dummy_drop += [i + '_' + str(df[i].unique()[-1])]

df = pd.get_dummies(df, columns = dummy_list)
df = df.drop(dummy_drop, axis = 1)

In [14]:
df['item_qty_sold']=df['Item_Outlet_Sales']/df['Item_MRP']

In [15]:
df.shape

(14204, 228)

In [16]:
train_len = len(train)
train_ = df[:train_len]
test_ = df[train_len:]

In [17]:
train_.shape

(8523, 228)

In [18]:
test_.shape

(5681, 228)

random forest - item_qty_sold 

In [19]:
from sklearn.model_selection import train_test_split,cross_val_score

In [20]:
yi=train_['item_qty_sold'].values.reshape(-1,1)
xi=train_.drop(['item_qty_sold','Item_Outlet_Sales'],axis=1)
Xi_train,Xi_test,yi_train,yi_test = train_test_split(xi,yi,test_size=0.2,random_state=42)

In [21]:
from sklearn.ensemble import RandomForestRegressor

In [22]:
Random_forest_reg_01=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=300, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=15,
                      min_samples_split=1000, min_weight_fraction_leaf=0.0,
                      n_estimators=190, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [23]:
Random_forest_reg_01.fit(Xi_train,yi_train.ravel())

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=300, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=15,
                      min_samples_split=1000, min_weight_fraction_leaf=0.0,
                      n_estimators=190, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [24]:
yi_pred = Random_forest_reg_01.predict(Xi_test)

In [25]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(yi_test, yi_pred))
print('Mean Squared Error:', metrics.mean_squared_error(yi_test, yi_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(yi_test, yi_pred)))

Mean Absolute Error: 5.212647857952508
Mean Squared Error: 46.613212140446784
Root Mean Squared Error: 6.827386918905855


In [26]:
test_.head()

,Item_Weight,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Visibility_per,Item_Fat_Content_Low Fat,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,...,Item_Identifier_num_52,Item_Identifier_num_53,Item_Identifier_num_54,Item_Identifier_num_55,Item_Identifier_num_56,Item_Identifier_num_57,Item_Identifier_num_58,Item_Identifier_num_59,Item_Identifier_num_60,item_qty_sold
0,20.750,107.8622,1999,NaN,0.756484,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,NaN
1,8.300,87.3198,2007,NaN,3.842768,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,14.600,241.7538,1998,NaN,9.957491,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,NaN
3,7.315,155.0340,2007,NaN,1.538839,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,NaN
4,12.792,234.2300,1985,NaN,11.859931,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [40]:
test_.drop(['item_qty_sold','Item_Outlet_Sales'],axis=1)

,Item_Weight,Item_MRP,Outlet_Establishment_Year,Item_Visibility_per,Item_Fat_Content_Low Fat,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Item_Identifier_num_51,Item_Identifier_num_52,Item_Identifier_num_53,Item_Identifier_num_54,Item_Identifier_num_55,Item_Identifier_num_56,Item_Identifier_num_57,Item_Identifier_num_58,Item_Identifier_num_59,Item_Identifier_num_60
0,20.750,107.8622,1999,0.756484,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,8.300,87.3198,2007,3.842768,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,14.600,241.7538,1998,9.957491,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,7.315,155.0340,2007,1.538839,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,12.792,234.2300,1985,11.859931,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5676,10.500,141.3154,1997,1.349647,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5677,7.600,169.1448,2009,14.299090,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5678,10.000,118.7440,2002,7.352856,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5679,15.300,214.6218,2007,0.000000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
test_.drop(['item_qty_sold','Item_Outlet_Sales'],axis=1,inplace=True)

C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [45]:
yi_pred_qty= Random_forest_reg_01.predict(test_)

In [46]:
test_.head()

,Item_Weight,Item_MRP,Outlet_Establishment_Year,Item_Visibility_per,Item_Fat_Content_Low Fat,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Item_Identifier_num_51,Item_Identifier_num_52,Item_Identifier_num_53,Item_Identifier_num_54,Item_Identifier_num_55,Item_Identifier_num_56,Item_Identifier_num_57,Item_Identifier_num_58,Item_Identifier_num_59,Item_Identifier_num_60
0,20.750,107.8622,1999,0.756484,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,8.300,87.3198,2007,3.842768,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,14.600,241.7538,1998,9.957491,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,7.315,155.0340,2007,1.538839,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,12.792,234.2300,1985,11.859931,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [47]:
yi_pred_qty

array([15.59535747, 15.64509897,  2.43553183, ..., 16.91592747,
       17.06190208, 16.70752594])

In [48]:
output = pd.DataFrame({'item_qty_sold': yi_pred_qty})

In [49]:
test_final = pd.concat([test, output], axis=1, sort=False)

In [50]:
test_final.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,item_qty_sold
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,15.595357
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1,15.645099
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store,2.435532
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1,16.696775
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3,26.635261


In [51]:
test_f = pd.concat([test_, output], axis=1, sort=False)

In [52]:
test_f.head()

,Item_Weight,Item_MRP,Outlet_Establishment_Year,Item_Visibility_per,Item_Fat_Content_Low Fat,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Item_Identifier_num_52,Item_Identifier_num_53,Item_Identifier_num_54,Item_Identifier_num_55,Item_Identifier_num_56,Item_Identifier_num_57,Item_Identifier_num_58,Item_Identifier_num_59,Item_Identifier_num_60,item_qty_sold
0,20.750,107.8622,1999,0.756484,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,15.595357
1,8.300,87.3198,2007,3.842768,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,15.645099
2,14.600,241.7538,1998,9.957491,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2.435532
3,7.315,155.0340,2007,1.538839,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,16.696775
4,12.792,234.2300,1985,11.859931,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,26.635261


random forest final

In [53]:
y=train_['Item_Outlet_Sales'].values.reshape(-1,1)
x=train_.drop(['Item_Outlet_Sales'],axis=1)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [54]:
rf_model = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=300, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=418, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [55]:
x.head()

,Item_Weight,Item_MRP,Outlet_Establishment_Year,Item_Visibility_per,Item_Fat_Content_Low Fat,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Item_Identifier_num_52,Item_Identifier_num_53,Item_Identifier_num_54,Item_Identifier_num_55,Item_Identifier_num_56,Item_Identifier_num_57,Item_Identifier_num_58,Item_Identifier_num_59,Item_Identifier_num_60,item_qty_sold
0,9.30,249.8092,1999,1.604730,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,14.951963
1,5.92,48.2692,2009,1.927822,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9.186454
2,17.50,141.6180,1999,1.676007,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14.809346
3,19.20,182.0950,1998,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.021967
4,8.93,53.8614,1987,0.000000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,18.467868


In [56]:
rf_model.fit(X_train,y_train.ravel())

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=300, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=418, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [57]:
y_pred = rf_model.predict(X_test)

In [58]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 23.451823489409346
Mean Squared Error: 4409.550952103455
Root Mean Squared Error: 66.40444979143683


In [59]:
final_preds = rf_model.predict(test_f)

In [60]:
final_preds

array([1667.03888782, 1371.30229638,  485.27580431, ..., 2042.28454   ,
       3709.06810922, 1323.8327552 ])

In [61]:
output_final = pd.DataFrame(final_preds)

In [62]:
output_final

,0
0,1667.038888
1,1371.302296
2,485.275804
3,2531.503076
4,6272.245684
...,...
5676,2394.417774
5677,2358.764598
5678,2042.284540
5679,3709.068109


In [63]:
test.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],
      dtype='object')

In [64]:
test.drop(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility','Item_Type', 'Item_MRP','Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],axis=1)

,Item_Identifier,Outlet_Identifier
0,FDW58,OUT049
1,FDW14,OUT017
2,NCN55,OUT010
3,FDQ58,OUT017
4,FDY38,OUT027
...,...,...
5676,FDB58,OUT046
5677,FDD47,OUT018
5678,NCO17,OUT045
5679,FDJ26,OUT017


In [66]:
 result_final = pd.concat([test, output_final], axis=1, sort=False)

In [67]:
result_final

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,0
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,1667.038888
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1,1371.302296
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store,485.275804
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1,2531.503076
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3,6272.245684
...,...,...,...,...,...,...,...,...,...,...,...,...
5676,FDB58,10.500,Regular,0.013496,Snack Foods,141.3154,OUT046,1997,Small,Tier 1,Supermarket Type1,2394.417774
5677,FDD47,7.600,Regular,0.142991,Starchy Foods,169.1448,OUT018,2009,Medium,Tier 3,Supermarket Type2,2358.764598
5678,NCO17,10.000,Low Fat,0.073529,Health and Hygiene,118.7440,OUT045,2002,NaN,Tier 2,Supermarket Type1,2042.284540
5679,FDJ26,15.300,Regular,0.000000,Canned,214.6218,OUT017,2007,NaN,Tier 2,Supermarket Type1,3709.068109


In [69]:
result_final.to_csv('submission_v2.csv', index=False)